In [5]:
import json
import json_coder
import pandas as pd
import networkx as nx
import befaas as bf
import befaas.logentry as le
from befaas.graph import build_function_graph, add_default_metadata, conv_to_ms

filepath = "../dumps/"
#logdumps = ["dump_streaming_google.json","dump_streaming_aws.json","dump_streaming_azure.json"]
#logdumps = ["dump_streaming_google.json","dump_streaming_aws.json"]
logdumps = ["dump_streaming_aws.json"]
#logdumps = ["dump_streaming_azure.json"]
outfile = "../img/2023_streaming_coldStarts.pkl"
plotdata = []

print("Everything is set up.")

Everything is set up.


In [6]:

debugpairs = []
for dump in logdumps:
    print(f"Include dump {dump} ...")
    print("Load dump... (may take a while)")
    data = bf.load_logs(filepath + dump)
    print("Done.")

    print("Sort data to contexts and find cold starts...")
    contexts = {}
    coldtsarts = []
    experimentStart = None

    for entry in data:
        id = entry.context_id
        if id == None:
            if isinstance(entry, le.ColdstartLog):
                coldtsarts.append(entry)
            else:
                print("Some other log entry type")

        if not id == None:
            if not (id in contexts):
                contexts[id] = []

            contexts[id].append(entry)
    print("Done.")

    print("Find xpairs...")
    xpairs = {}

    for ctx_id in contexts:
        ctx = contexts[ctx_id]
        for entry in ctx:
            id = entry.event["xPair"]
            if not (id in xpairs):
                xpairs[id] = []

            specialPair = False
            if isinstance(entry, le.PerfLog):
                if not entry.perf_type_data == "":
                    parts = entry.perf_type_data.split(":")
                    if len(parts) > 1:
                        id = parts[1]
                        if not (id in xpairs):
                            xpairs[id] = []
                        xpairs[id].append(entry)
                        specialPair = True

            if not specialPair:
                xpairs[id].append(entry)
    print("done.")

    errors = []

    for xpairID in xpairs:
        xpair = xpairs[xpairID]
        for entry in xpair:
            if isinstance(entry, le.ArtilleryLog):
                if experimentStart == None:
                    experimentStart = entry.timestamp
                if entry.timestamp < experimentStart:
                    experimentStart = entry.timestamp

    for xpairID in xpairs:
        xpair = xpairs[xpairID]

        requestStart = None
        requestEnd = None
        function = ""
        platform = ""

        if len(xpair) == 2:
            if str(xpair[0].url).__contains__("azure"):
                continue
                for entry in xpair:
                    if isinstance(entry, le.ArtilleryLog):
                        if entry.type == "before":
                           requestStart = entry.timestamp
                        if entry.type == "after":
                           requestEnd = entry.timestamp
                           function = str(entry.url).split("/")[-1]
                           platform = "azure"
            else:
                print("Error for " + xpair[0].url)
        else:
            for entry in xpair:
                if isinstance(entry, le.ArtilleryLog):
                    if entry.type == "before":
                       requestStart = entry.timestamp
                    if entry.type == "after":
                       requestEnd = entry.timestamp
                if isinstance(entry, le.RequestLog):
                    function = entry.function
                    platform = entry.platform

        latency = (requestEnd - requestStart).microseconds / 1000

        if not platform == "":
            row = {}
            row["Provider"] = platform
            row["function"] = function
            row["kind"] = "request"
            row["start"] = (requestStart - experimentStart).seconds
            row["latency"] = latency

            if row["start"] == 0:
                debugpairs.append(xpair)

            if row["start"] < 4000:
                plotdata.append(row)
            else:
                print("Error: " + json.dumps(row))
        else:
            debugpairs.append(xpair)

    print("Analyze cold starts...")

    # Find experiment Start for Cloud platform and use cloud timestamp for this
    experimentStart = None
    for cold in coldtsarts:
        if not cold.platform == "artillery":
            # print("Start is " + str(cold.timestamp))
            if experimentStart == None:
                    experimentStart = cold.timestamp
            if cold.timestamp < experimentStart:
                    experimentStart = cold.timestamp
    for cold in coldtsarts:
        if not cold.platform == "artillery":
            row = {}
            row["Provider"] = cold.platform
            row["function"] = cold.function
            row["kind"] = "coldstart"
            row["start"] = (cold.timestamp - experimentStart).seconds
            row["latency"] = 0

            if row["start"] < 4000:
                plotdata.append(row)
            else:
                print("Error: " + json.dumps(row))
                debugpairs.append(cold)


    print("Done.")

Include dump dump_streaming_google.json ...
Load dump... (may take a while)
Done.
Sort data to contexts and find cold starts...
Done.
Find xpairs...
done.
Analyze cold starts...
Done.
Include dump dump_streaming_aws.json ...
Load dump... (may take a while)
Done.
Sort data to contexts and find cold starts...
Done.
Find xpairs...
done.
Analyze cold starts...
Done.
Include dump dump_streaming_azure.json ...
Load dump... (may take a while)
Done.
Sort data to contexts and find cold starts...
Done.
Find xpairs...
done.
Analyze cold starts...
Error: {"Provider": "azure", "function": "addVideo", "kind": "coldstart", "start": 6830, "latency": 0}
Error: {"Provider": "azure", "function": "authDevice", "kind": "coldstart", "start": 6830, "latency": 0}
Error: {"Provider": "azure", "function": "getUserMeta", "kind": "coldstart", "start": 6830, "latency": 0}
Error: {"Provider": "azure", "function": "getVideos", "kind": "coldstart", "start": 6830, "latency": 0}
Error: {"Provider": "azure", "function":

In [7]:
df_calls = pd.DataFrame(plotdata)
print(df_calls.head())
df_calls.describe()

  Provider        function     kind  start  latency
0   google    registerUser  request      0    273.0
1   google  registerDevice  request      0    231.0
2   google  registerDevice  request      0     33.0
3   google    registerUser  request      0     27.0
4   google  registerDevice  request      0     34.0


,start,latency
count,40094.000000,40094.000000
mean,1248.624408,40.864044
std,762.276470,40.673841
min,0.000000,0.000000
25%,214.000000,24.000000
50%,1683.000000,32.000000
75%,1796.000000,46.000000
max,2208.000000,956.000000


In [8]:
df_calls.to_pickle(outfile)